In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch, ArrowStyle
from matplotlib.gridspec import GridSpecFromSubplotSpec
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from matplotlib.colors import to_rgb, Normalize
from matplotlib.lines import Line2D
from matplotlib.patheffects import withStroke
from matplotlib.colors import LinearSegmentedColormap

from matplotlib.cm import ScalarMappable

import networkx as nx

import os
import os.path as op

import importlib

import dgsp
import graph_examples as dgsp_graphs

import bimod_plots as plot

In [ ]:
path_to_figures = "./figures/Supplementary"

os.makedirs(path_to_figures, exist_ok=True)

inch_to_cm = 2.54
cm_to_inch = 1/inch_to_cm

colorpalette = ["#FFADAD", "#A0C4FF", "#CAFFBF", "#FFC6FF"]
palette_rgb = [to_rgb(color) for color in colorpalette]

colors_edges = np.array(["tab:red", "tab:blue", "tab:green", "tab:gray"])
markers = ["s", "o", "D", "^"]

os.makedirs(path_to_figures, exist_ok=True)

# Graph plots constants
edge_a=.2
edge_cs="arc3,rad=0.2"

In [ ]:
n_samples = 10
#orig_cmap = plt.get_cmap("Spectral", n_samples+1)
orig_cmap = plt.get_cmap("gist_rainbow", n_samples)
orig_cmap = plt.get_cmap("rainbow", n_samples)
#colorlist = ["gray"] + [orig_cmap(i) for i in range(n_samples//2)] + [orig_cmap(n_samples//2+i+1) for i in range(n_samples//2)]
colorlist = ["gray"] + [orig_cmap(i) for i in range(n_samples)]
custom_cmap = LinearSegmentedColormap.from_list("", colorlist)

fig, axes = plt.subplots(ncols=2, figsize=(10, 5))

axes[0].scatter([0, 1, 2, 3], [0]*4, c=palette_rgb, s=400, edgecolors="k", lw=2)
axes[0].scatter(np.linspace(0, 3, 9), [0.5]*9, c=np.arange(9), cmap=custom_cmap, s=400, edgecolors="k", lw=2)

axes[0].scatter(np.linspace(0, 3, 9), [0.25]*9, c=np.arange(9), cmap="RdBu", s=400)

n_rdbu = 11
RdBu = plt.get_cmap("RdBu", n_rdbu)
purple_def = orig_cmap(0)
purple_def = palette_rgb[-1]
#purple_def = "tab:purple"

color_offset = 1
RedPurpleBlue_list = [RdBu(color_offset), purple_def , RdBu(n_rdbu-color_offset-1)]
RedPurpleBlue_list = [RdBu(color_offset), RdBu(2*color_offset), purple_def , RdBu(n_rdbu-2*color_offset-1), RdBu(n_rdbu-color_offset-1)]
RedPurpleBlue = LinearSegmentedColormap.from_list("", RedPurpleBlue_list)

RedPurpleBlue = RedPurpleBlue.reversed()

n_plots = 50
axes[0].scatter(np.linspace(0, 3, n_plots), [0.12]*n_plots, c=np.arange(n_plots), cmap=RedPurpleBlue.resampled(n_plots), s=400)
axes[0].scatter(np.linspace(0, 3, n_plots), [0.35]*n_plots, c=np.arange(n_plots), cmap="tab20", s=400)

colorlist = np.array(palette_rgb)[[1, 2, 0]]
palette_cmap = LinearSegmentedColormap.from_list("", colorlist)
axes[0].scatter(np.linspace(0, 3, n_plots), [0.18]*n_plots, c=np.arange(n_plots), cmap=palette_cmap.resampled(n_plots), s=400)

axes[0].set_xlabel("Colors", fontsize=20)
axes[0].set_xticks([0, 1, 2, 3], ["Red", "Blue", "Green", "Purple"], fontsize=15)

axes[0].set_yticks([])

nclust = 9

for i in range(nclust):
    axes[1].scatter(i, 0.5, color=custom_cmap.resampled(nclust+1)(i+1), s=200, marker="s", lw=2)

axes[1].set_facecolor("gray")

exclude = [3, 4]
exclude = [4, 5]
exclude = [4]
cluster_colors8 = [custom_cmap.resampled(nclust+1)(i+1) for i in range(nclust) if i not in exclude]

cmap_8clusters = LinearSegmentedColormap.from_list("", ["gray"] + cluster_colors8)

for i in range(8):
    axes[1].scatter(i, 0.4, color=cluster_colors8[i], s=200, marker="s", lw=2)

axes[1].set_ylim(0.3, 0.6)

In [ ]:
gap_junc = False

wiring_sym = np.genfromtxt("./data/celegans_graph"+gap_junc*"_GAP"+".csv", delimiter=",")
neuron_df = pd.read_csv("./data/celegans_neurons.csv")

wiring_mod = dgsp.modularity_matrix(wiring_sym, null_model="outin")
print(f"Asymmetric wiring matrix has shape {wiring_sym.shape}")

nodes_labels = neuron_df.loc[:, "Neuron"]
nodes_posx = neuron_df.loc[:, "Position x"]
nodes_posy = neuron_df.loc[:, "Position y"]

d_mat = np.diag(wiring_sym.sum(axis=1))

U, S, Vh = dgsp.sorted_SVD(wiring_mod, fix_negative=False)
V = Vh.T

sort_idx = np.flip(np.argsort(S))
S = S[sort_idx]
U = U[:, sort_idx]
V = V[:, sort_idx]

directed_S = S

neuron_df.to_csv("./data/celegans_neurons.csv")
neuron_df

# Supplementaries

## Table of content:
- Canonical model
    - [Undirected Case](#)
    - [Undirected Case](#)
- *C. elegans* Proof of Concept
    - [Neurons Physical Location](#c-eleganss-neurons-physical-location)
    - [Source and Sink Neurons](#c-elegans-source-and-sink-neurons)
    - [Pathways](#c-elegans-pathways)
    - [Bicommunities for $k=9$](#c-elegans-bicommunities-for-k--9)
    - [Dissortative Embedding](#c-elegans-negative-bimodularity-embedding)
    - [Electric Gap Junction](#c-elegans-with-electric-gap)


## Canonical Modularity

In [ ]:
n_per_com = 40
density_value = .4
eig_id = 0

seed = 28

write_s = False
for_overleaf = True

graph = dgsp_graphs.toy_n_communities(n_per_com, 4, com_density=density_value,
                                      connect_density=[density_value, density_value, 0, 
                                                       0, density_value, density_value],
                                      #connect_density=[1, 1, 0, 1, 1, 1],
                                      connect_out_prob=[1, 0, 0, 0.5, 1, 0],
                                      seed=seed)

fig, axes = plt.subplots(ncols=3, figsize=(3*10, 10))

axes[0] = plot.plot_adjacency(graph, ax=axes[0], title_letter="A.", use_cmap=False,
                              override_title=r"Graph Adjacency Matrix $\mathbf{A}$")

axes[1] = plot.plot_adjacency(dgsp.configuration_null(graph, "outin"), ax=axes[1], title_letter="B.",
                              use_cmap=False, override_title=r"Configuration Null Model $\mathbf{Z}$")
axes[2] = plot.plot_adjacency(dgsp.modularity_matrix(graph, "outin"), ax=axes[2], title_letter="C.",
                              use_cmap=False, override_title=r"Modularity Matrix $\mathbf{B}$")

fig.savefig(op.join(path_to_figures, f"Suppl-Canonical-Modularity.png"), dpi=100, bbox_inches="tight")
fig.savefig(op.join(path_to_figures, f"Suppl-Canonical-Modularity.pdf"), dpi=300, bbox_inches="tight", format="pdf")

## Canonical Dissortative

In [ ]:
importlib.reload(plot)
n_per_com = 10

base_density = 0.8


fig, all_axes = plt.subplots(nrows=2, ncols=3, figsize=(30, 20))

#assort = False

assort_titles = ["Assortative Structure", "Dissortative Structure"]
for assort_i, assort in enumerate([True, False]):
    con_densities = 0.1 + 0.8 * assort
    #con_densities = 0.2 + 0.6 * assort

    dir_densities = 0.5

    graph = dgsp_graphs.toy_n_communities(n_per_com, 4, com_density=base_density*con_densities,
                                            connect_density=[base_density*con_densities,
                                                            base_density*(1-con_densities),
                                                            base_density*(1-con_densities),
                                                            base_density*(1-con_densities),
                                                            base_density*(1-con_densities),
                                                            base_density*con_densities],
                                            connect_out_prob=[0.5,
                                                            dir_densities,
                                                            dir_densities,
                                                            dir_densities,
                                                            dir_densities,
                                                            0.5],
                                            seed=1234)

    #axes.imshow(graph, cmap="binary_r", interpolation="none")

    plot.plot_adjacency(graph, all_axes[assort_i, 0], use_cmap=False, override_title="Graph Adjacency Matrix $\\mathbf{{A}}$")

    all_axes[assort_i, 0].set_xticks([])
    all_axes[assort_i, 0].set_yticks([])
    all_axes[assort_i, 0].set_ylabel(assort_titles[assort_i], fontsize=30)

    plot.plot_spectrum(graph, vector_id=0, ax=all_axes[assort_i, 1], fix_negative=False, write_s=True)

    U, S, Vh = dgsp.sorted_SVD(dgsp.modularity_matrix(graph))
    V = Vh.T

    angle = (V.T @ U)[0, 0]

    embed_ax = plot.plot_graph_embedding(graph, vector_id=0, ax=all_axes[assort_i, 2], use_cmap=False, directed_edges=False,
                                         edge_alpha=0.2, static_color="silver")
                                         #edge_alpha=0.2, static_color="silver", override_title=f"Embedding $(\\mathbf{{v}}_{{1}}^{{T}}\\mathbf{{u}}_{{1}}={angle:1.2f})$")

fig.savefig(op.join(path_to_figures, f"Suppl-Canonical-Dissortative.png"), dpi=100, bbox_inches="tight")
fig.savefig(op.join(path_to_figures, f"Suppl-Canonical-Dissortative.pdf"), dpi=300, bbox_inches="tight", format="pdf")

## Canonical Benchmarks (and Dissortative ?!)

In [ ]:
importlib.reload(dgsp)
importlib.reload(plot)

n_per_com = 40
density_value = .4
show_n_eig = 20
fontscale = 1

## Directed Fully Connected
base_density = 0.6
con_densities = 0.4
dir_densities = 0.5

fullycon = dgsp_graphs.toy_n_communities(n_per_com, 4, com_density=base_density,
                                        connect_density=[base_density, base_density*con_densities, base_density*con_densities,
                                                         base_density*con_densities, base_density*con_densities,
                                                         base_density],
                                        connect_out_prob=[0.5]*6,
                                                        seed=123)

## Block Cycle With Shortcuts
base_density = 0.6
con_densities = 0.7
dir_densities = 1

shortcut_cycle = dgsp_graphs.toy_n_communities(n_per_com, 4, com_density=base_density,
                                        connect_density=[base_density*con_densities, base_density*con_densities, base_density*con_densities,
                                                         base_density*con_densities, base_density*con_densities,
                                                         base_density*con_densities],
                                        connect_out_prob=[dir_densities, 0.5, 1-dir_densities,
                                                          dir_densities, 0.5,
                                                          dir_densities],
                                                          seed=28)

## Block Cycle With Random
base_density = 0.6
con_densities = 0.6
dir_densities = 1

random_prob = 0.05

random_cycle = dgsp_graphs.toy_n_communities(n_per_com, 4, com_density=base_density,
                                        connect_density=[base_density*con_densities, 0, base_density*con_densities,
                                                         base_density*con_densities, 0,
                                                         base_density*con_densities],
                                        connect_out_prob=[dir_densities, 0.5, 1-dir_densities,
                                                          dir_densities, 0.5,
                                                          dir_densities],
                                                          seed=28)

random_cycle[np.random.binomial(1, random_prob, random_cycle.shape) > 0] = 1

## Block Cycle With Missing Edges
base_density = 0.6
con_densities = 0.5
dir_densities = 1

missing_cycle = dgsp_graphs.toy_n_communities(n_per_com, 4, com_density=base_density,
                                        connect_density=[base_density*con_densities, 0, 0,
                                                         base_density*con_densities, 0,
                                                         base_density*con_densities],
                                        connect_out_prob=[dir_densities, 0.5, 1-dir_densities,
                                                          dir_densities, 0.5,
                                                          dir_densities],
                                                          seed=28)

graph_names = ["Conventional Communities", "Block Cycle (Shortcuts)", "Block Cycle (Noisy Edges)", "Disconnected Block Cycle"]
all_graphs = [fullycon, shortcut_cycle, random_cycle, missing_cycle]
all_kmean = [4, 12, 12, 7]
all_nrows = [2, 3, 3, 2]
all_ncols = [None, None, None, None]

fig, all_axes = plt.subplots(ncols=len(all_graphs), nrows=3, figsize=(10*len(all_graphs), 25),
                             gridspec_kw={"hspace":0.1, "height_ratios":[2, 1, 2]})

all_axes = np.atleast_2d(all_axes)

for graph_i, graph in enumerate(all_graphs):

    axes = all_axes[:, graph_i]

    U, S, Vh = dgsp.sorted_SVD(dgsp.modularity_matrix(graph, null_model="outin"))
    V = Vh.T

    n_nodes = graph.shape[0]

    vector_id_max = 2

    n_kmeans = all_kmean[graph_i]

    scale_factor = S

    edge_clusters, edge_clusters_mat = dgsp.edge_bicommunities(graph, U, V, vector_id_max, method="kmeans",
                                                            n_kmeans=n_kmeans, scale_S=scale_factor[:vector_id_max])
    n_clusters = np.max(edge_clusters)

    sending_communities, receiving_communities = dgsp.get_node_clusters(edge_clusters, edge_clusters_mat, scale=True)

    axes[0].imshow(edge_clusters_mat, cmap=cmap_8clusters.resampled(n_kmeans+1), interpolation="none", vmin=0, vmax=n_kmeans)

    for com_i in range(n_clusters):
        send = (edge_clusters_mat == com_i+1).sum(axis=1) > 0
        receive = (edge_clusters_mat == com_i+1).sum(axis=0) > 0

        plot_send = (np.where(send)[0].min(), np.where(send)[0].max())
        plot_receive = (np.where(receive)[0].min(), np.where(receive)[0].max())

        plot_send = np.where(send)[0]
        plot_receive = np.where(receive)[0]
        
        axes[0].scatter([-2-0.5-com_i]*len(plot_send), plot_send, marker="s", s=10, color=cmap_8clusters.resampled(n_kmeans+1)(com_i+1), edgecolors="none")
        axes[0].scatter(plot_receive, [-2-0.5-com_i]*len(plot_receive), marker="s", s=10, color=cmap_8clusters.resampled(n_kmeans+1)(com_i+1), edgecolors="none")

    axes[0].axis("off")

    plot.plot_spectrum(graph, vector_id=0, write_s=False, fix_negative=False, ax=axes[1])#, normalize_s=True)

    gs = GridSpecFromSubplotSpec(3, 1, axes[2], height_ratios=[1, 30, 2])
    axes[2].axis("off")

    ax_bicom = fig.add_subplot(gs[1])
    bimod_ids = dgsp.bimod_index_nodes(graph, sending_communities, receiving_communities, scale=True)

    #titles = [f"Cluster {i+1}, $Q_{{bi}}={bimod_ids[i]:1.2f}$" for i in range(len(bimod_ids))]
    titles = np.array([f"$C_{{{i+1}}}\,(Q_{{bi}}={bimod_ids[i]:1.2f})$" for i in range(len(bimod_ids))])

    sorts_ids = np.flip(np.argsort(bimod_ids))
    titles = titles[sorts_ids]
    sending_communities = sending_communities[sorts_ids]
    receiving_communities = receiving_communities[sorts_ids]

    titles = [""]*len(titles)

    plot.plot_all_bicommunity(graph, sending_communities, receiving_communities, fig=fig, axes=ax_bicom, #layout="",
                              draw_legend=False, nrows=all_nrows[graph_i], ncols=all_ncols[graph_i], titles=titles, cmap=RedPurpleBlue, edge_alpha=0.01)

    ## Titles

    axes[0].set_title(graph_names[graph_i], fontsize=30)
    axes[1].set_title("Spectrum", fontsize=30)
    #axes[2].set_title(f"Bicommunities $(k={all_kmean[graph_i]})$", fontsize=30, pad=-100)
    ax_bicom.set_title(f"Bicommunities $(k={all_kmean[graph_i]})$", fontsize=30, pad=-100)

    ## Colorbar
    xpos = axes[2].get_position().bounds[0]
    ypos = axes[2].get_position().bounds[1]
    xsize = axes[2].get_position().bounds[2]
    ysize = axes[2].get_position().bounds[3]

    #cmap_ax = fig.add_axes([xpos+2*xsize/5, ypos*0.9+ysize/2, xsize/5, ysize/30])
    cmap_ax = fig.add_axes([xpos + xsize/10, ypos*1.2, xsize*0.8, ysize/40])

    cmap_ax.spines[:].set_visible(False)

    norm = Normalize(vmin=-1, vmax=1)
    cbar = fig.colorbar(
        ScalarMappable(norm=norm, cmap=RedPurpleBlue),
        cax=cmap_ax,
        orientation="horizontal",
        ticks=np.linspace(-1, 1, 5),
    )
    cbar.ax.set_xticks([-1, 0, 1], labels=["Receive", "Both", "Send"])
    cbar.ax.tick_params(labelsize=20*fontscale)

    cmap_ax.scatter([-0.9], [0.5], color="none", s=60, edgecolor="k", lw=2, marker="D")
    cmap_ax.scatter([0], [0.5], color="none", s=60, edgecolor="k", lw=2, marker="o")
    cmap_ax.scatter([0.9], [0.5], color="none", s=60, edgecolor="k", lw=2, marker="s")

fig.savefig(op.join(path_to_figures, "Suppl-Canonical-Benchmark.png"), dpi=100, bbox_inches="tight")
fig.savefig(op.join(path_to_figures, "Suppl-Canonical-Benchmark.pdf"), dpi=300, bbox_inches="tight", format="pdf")

## Canonical Sources and Sinks

In [ ]:
importlib.reload(dgsp)

n_per_com = 40
base_density = 0.6

vector_id_max = 2
n_kmeans = 6

con_densities = 0.5
dir_densities = 1

binom_prob = 0.5

graph_orig = dgsp_graphs.toy_n_communities(n_per_com, 4, com_density=base_density,
                                    connect_density=[1,
                                                     0, 0, base_density, 0,
                                                     1],
                                    connect_out_prob=[1, 0, 0, 0.5, 0, 1]
                                    )

n_SourceSinks_max = 5

fig, axes = plt.subplots(nrows=2, ncols=n_SourceSinks_max, figsize=(n_SourceSinks_max*10, 20))
for i, n_SourceSinks in enumerate(np.arange(n_SourceSinks_max)):
    graph = graph_orig[n_per_com-n_SourceSinks:-n_per_com+n_SourceSinks][:, n_per_com-n_SourceSinks:-n_per_com+n_SourceSinks]

    if n_SourceSinks > 0:
        graph[:n_SourceSinks][:, :n_SourceSinks] = 0
        graph[-n_SourceSinks:][:, -n_SourceSinks:] = 0

    U, S, Vh = dgsp.sorted_SVD(dgsp.modularity_matrix(graph, null_model="outin"))
    V = Vh.T

    scale_factor = (S)[:vector_id_max]
    edge_clusters, edge_clusters_mat = dgsp.edge_bicommunities(graph, U, V, vector_id_max, method="kmeans",
                                                            n_kmeans=n_kmeans, scale_S=scale_factor)
    n_clusters = np.max(edge_clusters)

    sending_communities, receiving_communities = dgsp.get_node_clusters(edge_clusters, edge_clusters_mat, scale=True)

    node_clusters = np.array([1]*n_SourceSinks + [2]*n_per_com + [3]*n_per_com + [4]*n_SourceSinks)
    plot.plot_graph_embedding(graph, vector_id=0, fix_negative=False, ax=axes[0, i],
                              node_clusers=node_clusters,
                              override_title=f"Bimodular Embedding\n{n_SourceSinks} Sources and Sinks")
    # plot.plot_graph_embedding(graph, vector_id=1, fix_negative=False, ax=axes[1], node_clusers=node_clusters)

    axes[-1, i].set_title("Edge Clusters $(k=6)$", fontsize=30)
    axes[-1, i].imshow(edge_clusters_mat, cmap=cmap_8clusters.resampled(n_kmeans+1), interpolation="none", vmin=0, vmax=n_kmeans)

    # for com_i in range(n_clusters):
    #     send = (edge_clusters_mat == com_i+1).sum(axis=1) > 0
    #     receive = (edge_clusters_mat == com_i+1).sum(axis=0) > 0

    #     plot_send = (np.where(send)[0].min(), np.where(send)[0].max())
    #     plot_receive = (np.where(receive)[0].min(), np.where(receive)[0].max())

    #     plot_send = np.where(send)[0]
    #     plot_receive = np.where(receive)[0]
        
    #     axes[-1, i].scatter([-2-0.5-com_i]*len(plot_send), plot_send, marker="s", s=10, color=cmap_8clusters.resampled(n_kmeans+1)(com_i+1), edgecolors="none")
    #     axes[-1, i].scatter(plot_receive, [-2-0.5-com_i]*len(plot_receive), marker="s", s=10, color=cmap_8clusters.resampled(n_kmeans+1)(com_i+1), edgecolors="none")

    axes[-1, i].axis("off")

fig.savefig(op.join(path_to_figures, f"Suppl-Canonical-SourceSink-N{n_SourceSinks_max}.png"), dpi=100, bbox_inches="tight")
fig.savefig(op.join(path_to_figures, f"Suppl-Canonical-SourceSink-N{n_SourceSinks_max}.pdf"), dpi=300, bbox_inches="tight", format="pdf")

## Canonical Stability (wrt N_vec)

In [ ]:
from scipy.stats import pearsonr

importlib.reload(dgsp)

n_per_com = 40
base_density = 0.6

vector_id_max = 2
n_kmeans = 8

con_densities = 0.7
dir_densities = 1

graph = dgsp_graphs.toy_n_communities(n_per_com, 4, com_density=base_density,
                                    connect_density=[base_density*con_densities, 0, base_density*con_densities,
                                                    base_density*con_densities, 0,
                                                    base_density*con_densities
                                                    ],
                                    connect_out_prob=[dir_densities, 0.5, 1-dir_densities,
                                                        dir_densities, 0.5,
                                                        dir_densities
                                                        ])

U, S, Vh = dgsp.sorted_SVD(dgsp.modularity_matrix(graph, null_model="outin"))
V = Vh.T

scale_factor = (S**2)[:vector_id_max]
edge_clusters, edge_clusters_mat = dgsp.edge_bicommunities(graph, U, V, vector_id_max, method="kmeans",
                                                        n_kmeans=n_kmeans, scale_S=scale_factor)
n_clusters = np.max(edge_clusters)

sending_communities, receiving_communities = dgsp.get_node_clusters(edge_clusters, edge_clusters_mat, scale=True)
true_coms = np.hstack([sending_communities, receiving_communities])

n_trials = 100

scale_titles = ["No Scaling", "$\\sqrt{\\mu}$", "$\\mu$", "$\\mu^2$"]

all_vec_max = np.arange(2, 10, 1)

all_corr = np.zeros((len(all_vec_max), len(scale_titles), n_trials))
all_dice = np.zeros((len(all_vec_max), len(scale_titles), n_trials))

for vect_i, vector_id_max in enumerate(all_vec_max):
    for trial_i in range(n_trials):
        # Resets seed
        np.random.seed(None)
        graph = dgsp_graphs.toy_n_communities(n_per_com, 4, com_density=base_density,
                                            connect_density=[base_density*con_densities, 0, base_density*con_densities,
                                                            base_density*con_densities, 0,
                                                            base_density*con_densities
                                                            ],
                                            connect_out_prob=[dir_densities, 0.5, 1-dir_densities,
                                                                dir_densities, 0.5,
                                                                dir_densities
                                                                ])

        U, S, Vh = dgsp.sorted_SVD(dgsp.modularity_matrix(graph, null_model="outin"))
        V = Vh.T

        for scale_i, scale_factor in enumerate([None, np.sqrt(S)[:vector_id_max], (S)[:vector_id_max], (S**2)[:vector_id_max]]):
            edge_clusters, edge_clusters_mat = dgsp.edge_bicommunities(graph, U, V, vector_id_max, method="kmeans",
                                                                    n_kmeans=n_kmeans, scale_S=scale_factor)
            n_clusters = np.max(edge_clusters)

            sending_communities, receiving_communities = dgsp.get_node_clusters(edge_clusters, edge_clusters_mat, scale=True)


            com_vect = np.hstack([sending_communities, receiving_communities])

            confusion = np.zeros((len(true_coms), len(true_coms)))
            dice = np.zeros_like(confusion)
            for i, true in enumerate(true_coms):
                for j, test in enumerate(com_vect):
                    confusion[i, j] = pearsonr(true, test)[0]
                    dice[i, j] = 2 * ((true > 1e-10) * (test > 1e-10)).sum()/((true > 1e-10).sum() + (test > 1e-10).sum())

            all_dice[vect_i, scale_i, trial_i] = dice.max(axis=0).mean()
            all_corr[vect_i, scale_i, trial_i] = confusion.max(axis=0).mean()

In [ ]:
fig, axes = plt.subplots(figsize=(20, 10))

cmap_to_use = RedPurpleBlue.resampled(all_dice.shape[1])
# cmap_to_use = cmap_8clusters.resampled(all_dice.shape[1])
cmap_to_use = LinearSegmentedColormap.from_list("", palette_rgb).resampled(all_dice.shape[1]).reversed()

for i, between in enumerate(all_dice.swapaxes(0, 1)):
    axes.plot(all_vec_max, between.mean(axis=-1), lw=4, color=cmap_to_use(i))
    #axes.plot(all_vec_max, all_corr[:, i].mean(axis=-1), ls=":", lw=4, color=cmap_to_use(i))
    axes.fill_between(all_vec_max, np.percentile(between, 5, axis=-1), np.percentile(between, 95, axis=-1),
                      alpha=0.4, color=cmap_to_use(i), edgecolor="None")

axes.hlines(0.95, xmin=2, xmax=all_vec_max.max(), color="k", lw=2, ls=":")

axes.set_title("Stability of Bicommunities", fontsize=24)

axes.set_xlabel("Number of Components $N$", fontsize=22)
axes.set_ylabel("Dice Coefficient with True Communities", fontsize=22)
axes.tick_params(labelsize=20, width=2)

axes.spines[["top", "right"]].set_visible(False)
axes.spines[:].set_linewidth(2)

legend_titles = ["No Scaling", "Scale by $\\sqrt{{\\mu_{{n}}}}$", "Scale by $\\mu_{{n}}$", "Scale by $\\mu_{{n}}^2$"]
legend_elements = [Line2D([0], [0], lw=4, color=cmap_to_use(i), alpha=1, label=legend_titles[i]) for i in range(all_dice.shape[1])]

axes.legend(handles=legend_elements, fontsize=22)

fig.savefig(op.join(path_to_figures, f"Suppl-Canonical-Stability-{n_trials}trials.png"), dpi=100, bbox_inches="tight")
fig.savefig(op.join(path_to_figures, f"Suppl-Canonical-Stability-{n_trials}trials.pdf"), dpi=300, bbox_inches="tight", format="pdf")

## Canonical Stability (wrt Density)

In [ ]:
from scipy.stats import pearsonr

n_per_com = 40

overwrite = False

n_trials = 5
all_n_per_coms = np.arange(20, 120, 20)
base_densities = np.arange(0.1, 1, 0.05)
con_densities = np.arange(0.1, 1.1, 0.1)
#base_densities = np.hstack([np.arange(0.1, 0.3, 0.01), np.arange(0.3, 1, 0.1)])

if op.isfile(f"./data/Benchmark_all_dice-N{len(all_n_per_coms)}-B{len(base_densities)}-C{len(con_densities)}-T{n_trials}.npy") and not overwrite:
    all_dice = np.load(f"./data/Benchmark_all_dice-N{len(all_n_per_coms)}-B{len(base_densities)}-C{len(con_densities)}-T{n_trials}.npy")
else:
    all_dice = np.zeros((len(all_n_per_coms), len(con_densities), len(base_densities), n_trials))
    all_corr = np.zeros((len(all_n_per_coms), len(con_densities), len(base_densities), n_trials))

    for node_i, n_per_com in enumerate(all_n_per_coms):
        base_density = 0.6

        vector_id_max = 2
        n_kmeans = 8

        con_density = 0.7
        dir_densities = 1

        graph = dgsp_graphs.toy_n_communities(n_per_com, 4, com_density=base_density,
                                            connect_density=[base_density*con_density, 0, base_density*con_density,
                                                            base_density*con_density, 0,
                                                            base_density*con_density
                                                            ],
                                            connect_out_prob=[dir_densities, 0.5, 1-dir_densities,
                                                                dir_densities, 0.5,
                                                                dir_densities
                                                                ],
                                                                )#seed=28)

        U, S, Vh = dgsp.sorted_SVD(dgsp.modularity_matrix(graph, null_model="outin"))
        V = Vh.T

        scale_factor = S

        edge_clusters, edge_clusters_mat = dgsp.edge_bicommunities(graph, U, V, vector_id_max, method="kmeans",
                                                                n_kmeans=n_kmeans, scale_S=scale_factor[:vector_id_max])
        n_clusters = np.max(edge_clusters)

        sending_communities, receiving_communities = dgsp.get_node_clusters(edge_clusters, edge_clusters_mat, scale=True)

        true_coms = np.hstack([sending_communities, receiving_communities])
        dir_densities = 1

        for dense_i, base_density in enumerate(base_densities):
            for con_i, con_density in enumerate(con_densities):
                for trial_i in range(n_trials):
                    graph = dgsp_graphs.toy_n_communities(n_per_com, 4, com_density=base_density,
                                                        connect_density=[base_density*con_density, 0, base_density*con_density,
                                                                        base_density*con_density, 0,
                                                                        base_density*con_density
                                                                        ],
                                                        connect_out_prob=[dir_densities, 0.5, 1-dir_densities,
                                                                            dir_densities, 0.5,
                                                                            dir_densities
                                                                            ])

                    U, S, Vh = dgsp.sorted_SVD(dgsp.modularity_matrix(graph, null_model="outin"))
                    V = Vh.T

                    scale_factor = S

                    edge_clusters, edge_clusters_mat = dgsp.edge_bicommunities(graph, U, V, vector_id_max, method="kmeans",
                                                                            n_kmeans=n_kmeans, scale_S=scale_factor[:vector_id_max])
                    n_clusters = np.max(edge_clusters)

                    sending_communities, receiving_communities = dgsp.get_node_clusters(edge_clusters, edge_clusters_mat, scale=True)

                    com_vect = np.hstack([sending_communities, receiving_communities])

                    # fig, axes = plt.subplots(ncols=2, figsize=(10, 5))

                    #confusion = np.zeros((len(true_coms), len(true_coms)))
                    dice = np.zeros((len(true_coms), len(true_coms)))
                    for i, true in enumerate(true_coms):
                        for j, test in enumerate(com_vect):
                            #confusion[i, j] = pearsonr(true, test)[0]
                            dice[i, j] = 2 * ((true > 1e-10) * (test > 1e-10)).sum()/((true > 1e-10).sum() + (test > 1e-10).sum())

                    all_dice[node_i, con_i, dense_i, trial_i] = dice.max(axis=0).mean()
                    #all_corr[node_i, dense_i, trial_i] = confusion.max(axis=0).mean()

    np.save(f"./data/Benchmark_all_dice-N{len(all_n_per_coms)}-B{len(base_densities)}-C{len(con_densities)}-T{n_trials}.npy", all_dice)

In [ ]:
fig, axes = plt.subplots(figsize=(10, 5))

axes.plot(base_densities, all_dice.mean(axis=(1, -1)).T)
#axes.fill_between(base_densities, all_corr.max(axis=1), all_corr.min(axis=1), alpha=0.6)

axes.set_title("Stability of Bicommunities", fontsize=24)

axes.set_xlabel("Density of Edges", fontsize=22)
axes.set_ylabel("Dice Coefficient with True Communities", fontsize=22)
axes.tick_params(labelsize=20, width=2)

axes.spines[["top", "right"]].set_visible(False)
axes.spines[:].set_linewidth(2)

axes.legend([f"$N={4*n_per_com}$ Nodes" for n_per_com in all_n_per_coms], fontsize=12, ncols=2)

In [ ]:
fig, axes = plt.subplots(figsize=(10, 10))

axes.imshow(all_dice.mean(axis=(0, -1)).T, cmap="turbo", vmin=0.5, vmax=1, aspect="auto")

axes.contour(all_dice.mean(axis=(0, -1)).T, [0.95], extent=[-0.5, len(con_densities)-0.5, -0.5, len(base_densities)-0.5], linewidths=4, cmap="binary_r")
#axes.fill_between(base_densities, all_corr.max(axis=1), all_corr.min(axis=1), alpha=0.6)

axes.set_title("Stability of Bicommunities (Dice)", fontsize=24)

axes.set_xlabel("Density of Connecting Edges", fontsize=22)
axes.set_xticks(np.arange(all_dice.mean(axis=(0, -1)).T.shape[1]), labels=[f"{lab:1.2f}" for lab in con_densities])
axes.set_ylabel("Density of Community Edges", fontsize=22)
axes.set_yticks(np.arange(all_dice.mean(axis=(0, -1)).T.shape[0]), labels=[f"{lab:1.2f}" for lab in base_densities])

axes.tick_params(labelsize=18, width=2)

plot.add_cbar(fig, axes)

## Canonical Undirected (BICOM)

In [ ]:
importlib.reload(dgsp)
importlib.reload(plot)

n_per_com = 40
density_value = .4
show_n_eig = 20
fontscale = 1

n_ideal_dir = 0
vector_id = 0

#n_per_com = 10

base_density = 0.6

assort = False

con_densities = 0.4

dir_densities = 0.5

graph = dgsp_graphs.toy_n_communities(n_per_com, 4, com_density=base_density,
                                        connect_density=[base_density, base_density*con_densities, base_density*con_densities,
                                                         base_density*con_densities, base_density*con_densities,
                                                         base_density],
                                        connect_out_prob=[0.5, dir_densities, dir_densities,
                                                        dir_densities, dir_densities,
                                                        0.5],
                                                        seed=123)

graph += graph.T

U, S, Vh = dgsp.sorted_SVD(dgsp.modularity_matrix(graph, null_model="outin"))
V = Vh.T

n_nodes = graph.shape[0]

vector_id_max = 2
n_kmeans = 4

scale_factor = S

edge_clusters, edge_clusters_mat = dgsp.edge_bicommunities(graph, U, V, vector_id_max, method="kmeans",
                                                           n_kmeans=n_kmeans, scale_S=scale_factor[:vector_id_max])
n_clusters = np.max(edge_clusters)

sending_communities, receiving_communities = dgsp.get_node_clusters(edge_clusters, edge_clusters_mat, scale=True)

#fig, axes = plt.subplots(nrows=2, figsize=(10, 20), gridspec_kw={"wspace":0.1})
fig, axes = plt.subplots(nrows=2, figsize=(10, 20), gridspec_kw={"hspace":0})

# Edge colors
graph_pos = {i: (U[i, vector_id], V[i, vector_id]) for i in range(n_nodes)}
labels = {i: "" for i in range(n_nodes)}

edge_list = nx.DiGraph(graph).edges()

axes[0].imshow(edge_clusters_mat, cmap=cmap_8clusters.resampled(n_kmeans+1), interpolation="none", vmin=0, vmax=n_kmeans)

for com_i in range(n_clusters):
    send = (edge_clusters_mat == com_i+1).sum(axis=1) > 0
    receive = (edge_clusters_mat == com_i+1).sum(axis=0) > 0

    plot_send = (np.where(send)[0].min(), np.where(send)[0].max())
    plot_receive = (np.where(receive)[0].min(), np.where(receive)[0].max())

    plot_send = np.where(send)[0]
    plot_receive = np.where(receive)[0]
    
    axes[0].scatter([-2-0.5-com_i]*len(plot_send), plot_send, marker="s", s=10, color=cmap_8clusters.resampled(n_kmeans+1)(com_i+1), edgecolors="none")
    axes[0].scatter(plot_receive, [-2-0.5-com_i]*len(plot_receive), marker="s", s=10, color=cmap_8clusters.resampled(n_kmeans+1)(com_i+1), edgecolors="none")

axes[0].axis("off")

gs = GridSpecFromSubplotSpec(3, 1, axes[1], height_ratios=[1, 20, 2])
axes[1].axis("off")

ax_bicom = fig.add_subplot(gs[1])
bimod_ids = dgsp.bimod_index_nodes(graph, sending_communities, receiving_communities, scale=True)

titles = [f"Cluster {i+1}, $Q_{{bi}}={bimod_ids[i]:1.2f}$" for i in range(len(bimod_ids))]
plot.plot_all_bicommunity(graph, sending_communities, receiving_communities, fig=fig, axes=ax_bicom, #layout="",
                          draw_legend=False, nrows=2, titles=titles, cmap=RedPurpleBlue)

## Colorbar
xpos = axes[1].get_position().bounds[0]
ypos = axes[1].get_position().bounds[1]
xsize = axes[1].get_position().bounds[2]
ysize = axes[1].get_position().bounds[3]

#cmap_ax = fig.add_axes([xpos+2*xsize/5, ypos*0.9+ysize/2, xsize/5, ysize/30])
cmap_ax = fig.add_axes([xpos + xsize/10, ypos*1.2, xsize*0.8, ysize/20])

cmap_ax.spines[:].set_visible(False)

norm = Normalize(vmin=-1, vmax=1)
cbar = fig.colorbar(
    ScalarMappable(norm=norm, cmap=RedPurpleBlue),
    cax=cmap_ax,
    orientation="horizontal",
    ticks=np.linspace(-1, 1, 5),
)
cbar.ax.set_xticks([-1, 0, 1], labels=["Receive", "Both", "Send"])
cbar.ax.tick_params(labelsize=20*fontscale)

cmap_ax.scatter([-0.9], [0.5], color="none", s=180, edgecolor="k", lw=2, marker="D")
cmap_ax.scatter([0], [0.5], color="none", s=180, edgecolor="k", lw=2, marker="o")
cmap_ax.scatter([0.9], [0.5], color="none", s=180, edgecolor="k", lw=2, marker="s")

# ax_bicom_2 = fig.add_subplot(gs_all_coms[2])
# plot.plot_all_bicommunity(graph, sending_communities, receiving_communities, fig=fig, axes=ax_bicom_2, draw_legend=False)

# fig.savefig(op.join(path_to_figures, f"Suppl-CElegans-Position-eig{eig_id:02d}.png"), dpi=100, bbox_inches="tight")
#fig.savefig(op.join(path_to_figures, "001-bicommunities-toy.png"), bbox_inches="tight")

## Canonical Undirected

In [ ]:
n_per_com = 40
rand_density = 0

con_densities = 0.5
dir_densities = 1

fig, axes = plt.subplots(ncols=2, nrows=3, figsize=(20, 25), gridspec_kw={"height_ratios":[2, 1, 2]})

graph = dgsp_graphs.toy_n_communities(n_per_com, 4, com_density=con_densities,
                                      connect_density=[
                                          con_densities, con_densities, 0,
                                          0, con_densities,
                                          con_densities
                                          ],
                                      connect_out_prob=[
                                          dir_densities, 1-dir_densities, 1-dir_densities,
                                          0.5, dir_densities,
                                          1-dir_densities
                                          ],
                                          seed=1234)

matrix = graph + graph.T

plot.plot_adjacency(graph, ax=axes[0, 0], use_cmap=True, override_title="$\mathbf{{A}}$")
plot.plot_adjacency(matrix, ax=axes[0, 1], use_cmap=True, override_title="$\mathbf{{A}}+\mathbf{{A}}^T$")

plot.plot_spectrum(graph, ax=axes[1, 0], fix_negative=False)
plot.plot_spectrum(matrix, ax=axes[1, 1], fix_negative=False)

plot.plot_graph_embedding(graph, ax=axes[2, 0], use_cmap=True, directed_edges=False)

#plot.plot_graph_embedding(graph+graph.T, ax=axes[1, 1], use_cmap=True, directed_edges=False)

U, S, Vh = dgsp.sorted_SVD(dgsp.modularity_matrix(matrix))
V = Vh.T

ax = axes[2, 1]
fontscale = 1.2
vector_id = 0
second_vector = 2

n_nodes = graph.shape[0]

graph_pos = {i: (U[i, vector_id], U[i, vector_id+second_vector]) for i in range(n_nodes)}
labels = {i: "" for i in range(n_nodes)}

if ax is None:
    _, ax = plt.subplots(figsize=(8, 8))

nx_graph = nx.Graph(matrix)

nx.draw_networkx_edges(nx_graph, pos=graph_pos, alpha=0.02, ax=ax)

colors = [palette_rgb[i // n_per_com] for i in np.arange(n_nodes)]

ax.scatter(
    U[:, vector_id],
    U[:, vector_id+second_vector],
    s=200,
    color=colors,
    edgecolor="k",
    linewidth=2,
    zorder=2,
)

# Parameters

ax.set_title(f"Modularity Embedding", fontsize=20*fontscale)
ax.tick_params(
    left=True,
    bottom=True,
    labelleft=True,
    labelbottom=True,
    labelsize=16 * fontscale,
    width=2,
)
ax.spines[["top", "right"]].set_visible(False)
ax.spines[:].set_linewidth(2)

ax.set_xticks(
    np.linspace(-0.1, 0.1, 3),
    labels=np.linspace(-0.1, 0.1, 3),
    fontsize=18 * fontscale,
)
ax.set_yticks(
    np.linspace(-0.1, 0.1, 3),
    labels=np.linspace(-0.1, 0.1, 3),
    fontsize=18 * fontscale,
)

ax.set_xlabel(
    f"$1^{{st}}$ Eigenvector $\\mathbf{{u}}_{{{vector_id+1}}}$",
    fontsize=18 * fontscale,
)
ax.set_ylabel(
    f"$2^{{nd}}$ Eigenvector $\\mathbf{{u}}_{{{vector_id+1+second_vector}}}$",
    fontsize=18 * fontscale,
)

## Less ideal synthetic model

In [ ]:
importlib.reload(plot)

n_per_com = 40
rand_density = 0

con_densities = [0.2, 0.4, 0.5, 0.6, 0.8]
dir_densities = [0.5, 0.6, 0.7, 0.8, 1]

eig_id = 0
# HERE
fontscale = 1.2

write_s = True
for_overleaf = True

fig, axes = plt.subplots(len(con_densities), len(dir_densities),
                         gridspec_kw={"hspace": 0, "wspace": 0},
                         #gridspec_kw={"hspace": 0.08, "wspace": 0.08},
                         figsize=(len(con_densities)*8, len(dir_densities)*8))#,
                         #sharex=True, sharey=True)

for row_i, ax_row in enumerate(axes):
    for col_i, ax in enumerate(ax_row):
        # Building the graph
        graph = dgsp_graphs.toy_n_communities(n_per_com, 4, com_density=con_densities[row_i],
                                              connect_density=[con_densities[row_i], con_densities[row_i], 0,
                                                               0, con_densities[row_i],
                                                               con_densities[row_i]],
                                              connect_out_prob=[dir_densities[col_i], 1-dir_densities[col_i], 1-dir_densities[col_i],
                                                                0.5, dir_densities[col_i], 1-dir_densities[col_i]],
                                              seed=1234)
        
        graph += dgsp_graphs.random_graph(n_per_com*4, rand_density, seed=1234)
        graph = (graph > 0).astype(int)

        title = ""
        if row_i == 0:
            title += f"$\gamma_{{dir}}={dir_densities[col_i]}$"

        ax = plot.plot_graph_embedding(graph, vector_id=eig_id, write_label=True, write_var=True,
                                       ax=ax, directed_edges=False,
                                       override_title=title, fontscale=fontscale, label_lw=6)

        ax.spines[:].set_visible(True)

        ax.set_xlabel("")
        ax.set_ylabel("")
        
        if col_i == 0:
            ax.set_ylabel(f"$\gamma_{{con}}={con_densities[row_i]}$", fontsize=22*fontscale)

        ax.set_yticks([])
        ax.set_xticks([])

# fig.savefig(op.join(path_to_figures, f"Suppl-Canonical-LessIdeal-r{rand_density:1.1f}-Overleaf.png"), dpi=100, bbox_inches="tight")

## Assortiative to Dissortative and Undirected to Directed

In [ ]:
importlib.reload(plot)

n_per_com = 40
rand_density = 0

base_density = 0.8
#con_densities = [0.2, 0.4, 0.5, 0.6, 0.8]
con_densities = [0.8, 0.6, 0.5, 0.4, 0.2]
dir_densities = [0.5, 0.6, 0.7, 0.8, 1]

vector_id = 0
# HERE
fontscale = 1.2

write_s = True
for_overleaf = True

fig, axes = plt.subplots(len(con_densities), len(dir_densities),
                         gridspec_kw={"hspace": 0.01, "wspace": 0.01},
                         #gridspec_kw={"hspace": 0.08, "wspace": 0.08},
                         figsize=(len(con_densities)*8, len(dir_densities)*8))#,
                         #sharex=True, sharey=True)

for row_i, ax_row in enumerate(axes):
    for col_i, ax in enumerate(ax_row):
        # Building the graph
        graph = dgsp_graphs.toy_n_communities(n_per_com, 4, com_density=base_density*con_densities[row_i],
                                              connect_density=[base_density*con_densities[row_i],
                                                               base_density*(1-con_densities[row_i]),
                                                               base_density*(1-con_densities[row_i]),
                                                               base_density*(1-con_densities[row_i]),
                                                               base_density*(1-con_densities[row_i]),
                                                               base_density*con_densities[row_i]],
                                              connect_out_prob=[0.5,
                                                                dir_densities[col_i],
                                                                dir_densities[col_i],
                                                                dir_densities[col_i],
                                                                dir_densities[col_i],
                                                                0.5],
                                              seed=1234)

        title = ""
        if row_i == 0:
            title += f"$\gamma_{{dir}}={dir_densities[col_i]}$"

        ax = plot.plot_graph_embedding(graph, vector_id=eig_id, n_com=2,
                                       ax=ax, directed_edges=False, write_var=True,
                                       override_title=title, fontscale=fontscale, label_lw=6, fix_negative=False)
        #ax = plot.plot_adjacency(graph, ax=ax, title_letter="", use_cmap=False, override_title=title)
        #U, S, Vh = dgsp.sorted_SVD(dgsp.modularity_matrix(graph), fix_negative=False)
        #ax.plot(S)

        ax.spines[:].set_visible(True)

        ax.set_xlabel("")
        ax.set_ylabel("")
        
        if col_i == 0:
            ax.set_ylabel(f"$\gamma_{{con}}={con_densities[row_i]}$", fontsize=22*fontscale)

        ax.set_yticks([])
        ax.set_xticks([])

# fig.savefig(op.join(path_to_figures, f"Suppl-Canonical-AssorDissor-Overleaf.png"), dpi=100, bbox_inches="tight")

## *C. elegans*

## *C. elegans*'s Neurons Physical Location

In [ ]:
# CElegans-Position-eig00

fontscale = 1.2

n_cmap_points = 100
alpha_perc = 20
FAST = False

fig, axes = plt.subplots(figsize=(15, 10))

eig_id = np.where(S > 1e-10)[0][0]
#eig_id = 1

graph = nx.DiGraph(wiring_sym)
graph_pos = {i: (U[i, eig_id], V[i, eig_id]) for i in range(wiring_sym.shape[0])}

edge_colors = np.zeros_like(np.array(graph.edges())[:, 0])

for i, (_, target) in enumerate(graph.edges()):
    edge_colors[i] = neuron_df[neuron_df["Neuron"] == nodes_labels[target]]["Type_num"].values[0]
    
colors = palette_rgb

#nx.draw_networkx_edges(graph, pos=graph_pos, alpha=.05)
if not FAST:
    nx.draw_networkx_edges(graph, pos=graph_pos, edge_color=colors_edges[edge_colors],
                           alpha=edge_a, connectionstyle=edge_cs)

x_norm = nodes_posx + 0.4
y_norm = (nodes_posy - nodes_posy.min())
ap_angle = np.arctan2(y_norm/np.abs(y_norm).max(), x_norm/np.abs(x_norm).max())
ap_angle = n_cmap_points*(ap_angle - ap_angle.min())/(ap_angle.max() - ap_angle.min())

angle_cmap = plt.get_cmap("rainbow_r", n_cmap_points)
#angle_cmap = Palette_cmap.resampled(n_cmap_points)
ap_colors = np.array([angle_cmap(a) for a in ap_angle.astype(int)])

#axes_pos = [0.51, 0.128, 0.38, 0.22]
#axes_pos = [0.14, 0.128, 0.21, 0.11]
if eig_id == 1:
    axes_pos = [0.12, 0.128, 0.32, 0.16]
else:
    axes_pos = [0.51, 0.128, 0.38, 0.22]

#display_axes = fig.add_axes(axes_pos, facecolor="w")
display_axes = fig.add_axes(axes_pos, facecolor="none")
display_axes.spines[:].set_visible(False)
display_axes.text(0.5, 0, "Head", fontsize=16, ha="right")
display_axes.text(-0.5, 0, "Tail", fontsize=16)
graph_pos = {i: (x_norm[i], y_norm[i]) for i in range(wiring_sym.shape[0])}

if not FAST:
    nx.draw_networkx_edges(nx.Graph(wiring_sym), pos=graph_pos, alpha=.1, ax=display_axes,
                           edge_color=colors_edges[edge_colors], edgelist=graph.edges())

true_types = ["SENSORY NEURONS", "INTERNEURONS", "MOTOR NEURONS", "SEX SPECIFIC"]
#for i, n_type in enumerate(neuron_df["Type"].unique()):
for i, n_type in enumerate(true_types):
    neurons_in_type = neuron_df.loc[neuron_df["Type"] == n_type, "Neuron"]

    #type_mask = np.array([lab in neurons_in_type.values for lab in bullmore_label])
    type_mask = np.array([lab in neurons_in_type.values for lab in nodes_labels])
    #axes.scatter(U[type_mask, eig_id], V[type_mask, eig_id], color=colors[i], s=80, label=n_type,
    #             zorder=3, edgecolor="k", lw=1, alpha=1, marker=markers[i])
    axes.scatter(U[type_mask, eig_id], V[type_mask, eig_id], color=ap_colors[type_mask], s=80, label=n_type,
                 zorder=3, edgecolor="k", lw=1, alpha=1, marker=markers[i])
    
    display_axes.scatter(x_norm[type_mask], y_norm[type_mask], color=ap_colors[type_mask],
                         s=50, edgecolor="k", lw=1, marker=markers[i])

#label_filter = ["AVAL", "AVAR", "AVEL", "AVER", "RIAR", "RIAL"]
label_filter = ["PHAL", "PHAR", "PHBL", "PHBR", "PHCR", "PHCL", "PLMR", "PLML"]
#for i, neuron in enumerate(bullmore_label):

squared_rad = U[:, eig_id] ** 2 + V[:, eig_id] ** 2
by_radius = np.argsort(squared_rad)
alpha_vector = (squared_rad > np.percentile(squared_rad, 100-alpha_perc)).astype(int)

#for i, neuron in enumerate(nodes_labels):
for i in by_radius:
    #if neuron in label_filter:
    #if nodes_labels[i] in label_filter:
    if True:
        #axes.text(1.01*U[i, eig_id], 1.01*V[i, eig_id], neuron, fontsize=16, alpha=alpha_vector[i],
        axes.text(1.01*U[i, eig_id], 1.01*V[i, eig_id], nodes_labels[i], fontsize=16, alpha=alpha_vector[i],
                  path_effects=[
                      withStroke(
                          linewidth=4,
                          foreground="w",
                          alpha=0.8*alpha_vector[i]
                          )])

x_max, y_max = np.max(np.vstack([U[:, eig_id], V[:, eig_id]]), axis=1)
x_min, y_min = np.min(np.vstack([U[:, eig_id], V[:, eig_id]]), axis=1)

abs_min = max([x_min, y_min])
abs_max = min([x_max, y_max])

#axes.plot([abs_min, abs_max], [abs_min, abs_max], color="tab:orange")
axes.plot([0, 0], [y_min, y_max], ls=":", lw=2, color="k", zorder=0)
axes.plot([x_min, x_max], [0, 0], ls=":", lw=2, color="k", zorder=0)


true_types = ["SENSORY NEURONS", "INTERNEURONS", "MOTOR NEURONS", "SEX SPECIFIC"]
legend_elements = [Line2D([0], [0], color="w", alpha=1, markersize=10, markeredgecolor="k",
                          markeredgewidth=2, marker=markers[i], label=type) for i, type in enumerate(true_types)]

#all_axes.legend(handles=legend_elements, loc="lower right", fontsize=16, title="Connection density")
axes.legend(handles=legend_elements, fontsize=18, loc="upper left")

#axes.set_title(f"Bimodularity Embedding", fontsize=22 * fontscale)
#axes[1].set_title(f"Bimodularity embedding $s_{{{eig_id+1}}}={S[eig_id]:1.2f}$", fontsize=22 * fontscale)
axes.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True, labelsize=16 * fontscale)
axes.set_xticks(np.linspace(-0.2, .4, 4))
axes.set_yticks(np.linspace(-.1, .2, 4))
axes.set_xlabel(f"{eig_id + 1}th Left Singular Vector ($\\mathbf{{u}}_{{{eig_id+1}}}$)", fontsize=18 * fontscale)
axes.set_ylabel(f"{eig_id + 1}th Right Singular Vector ($\\mathbf{{v}}_{{{eig_id+1}}}$)", fontsize=18 * fontscale)
axes.spines[["top", "right"]].set_visible(False)

prev_xlim = axes.get_xlim()
prev_ylim = axes.get_ylim()

display_axes.set_xticks([])
_ = display_axes.set_yticks([])

fig.savefig(op.join(path_to_figures, f"Suppl-CElegans-Position-eig{eig_id:02d}.png"), dpi=100, bbox_inches="tight")
fig.savefig(op.join(path_to_figures, f"Suppl-CElegans-Position-eig{eig_id:02d}.pdf"), dpi=300, bbox_inches="tight", format="pdf")

## *C. elegans* Source and Sink Neurons

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(30, 20),
                         #sharex="row", sharey="row",
                         gridspec_kw={"wspace":0.12, "hspace":0.15})

eig_id = 0

zero_deg_thresh = 1
eig_thresh = 0
#min_con = 3

titles = [f"Right source ($k_{{in}}\leq {zero_deg_thresh},\,\\mathbf{{u}}_{{{eig_id+1}}}>{eig_thresh}$)",
          f"Left source ($k_{{in}}\leq {zero_deg_thresh},\,\\mathbf{{u}}_{{{eig_id+1}}}<{eig_thresh}$)",
          f"Upper sink ($k_{{out}}\leq {zero_deg_thresh},\,\\mathbf{{v}}_{{{eig_id+1}}}>{eig_thresh}$)",
          f"Lower sink ($k_{{out}}\leq {zero_deg_thresh},\,\\mathbf{{v}}_{{{eig_id+1}}}<{eig_thresh}$)"]

true_types = ["SENSORY NEURONS", "INTERNEURONS", "MOTOR NEURONS", "SEX SPECIFIC"]
legend_elements = [Line2D([0], [0], color="w", alpha=1, markersize=15, markeredgecolor="k",
                          markeredgewidth=2, marker=markers[i], label=type) for i, type in enumerate(true_types)]

for ax_i, ax_col in enumerate(axes.T):
    for ax_j, ax in enumerate(ax_col):

        wiring_red = wiring_sym.copy()
        if ax_j == 0:
            sink_source = np.logical_and(wiring_red.sum(axis=ax_i) <= zero_deg_thresh,
                                         [U, V][ax_i][:, eig_id] > eig_thresh, dtype=bool)
            #sink_source = np.logical_and(sink_source, wiring_red.sum(axis=1-ax_i) > min_con, dtype=bool)
        else:
            sink_source = np.logical_and(wiring_red.sum(axis=ax_i) <= zero_deg_thresh,
                                         [U, V][ax_i][:, eig_id] < -eig_thresh, dtype=bool)
            #sink_source = np.logical_and(sink_source, wiring_red.sum(axis=1-ax_i) > min_con, dtype=bool)

        mask = (wiring_red[sink_source].sum(axis=0) > 0) + (wiring_red[:, sink_source].sum(axis=1) > 0)
        mask = (mask + sink_source).astype(bool)

        wiring_red = wiring_red[mask][:, mask]
        labels_red = nodes_labels[mask].to_list()
        U_red = U[mask]
        V_red = V[mask]

        color_red = ap_colors[mask]

        graph = nx.DiGraph(wiring_red)
        graph_pos = {i: (U_red[i, eig_id], V_red[i, eig_id]) for i in range(wiring_red.shape[0])}
        
        mycmap = plt.get_cmap("tab20")
        colors_edge = np.array(["tab:red", "tab:blue", "tab:green", "tab:gray"])
        colors = np.array(palette_rgb)

        edge_colors = np.zeros_like(np.array(graph.edges())[:, 0])

        for i, (_, target) in enumerate(graph.edges()):
            edge_colors[i] = neuron_df[neuron_df["Neuron"] == labels_red[target]]["Type_num"].values[0]
        #nx.draw_networkx_edges(graph, pos=graph_pos, alpha=.1, ax=ax)
        nx.draw_networkx_edges(graph, pos=graph_pos, edge_color=colors_edge[edge_colors],
                               alpha=2*edge_a, ax=ax, connectionstyle=edge_cs)

        ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)

        for i, n_type in enumerate(true_types): 
        # for i, n_type in enumerate(neuron_df["Type"].unique()):
            neurons_in_type = neuron_df.loc[neuron_df["Type"] == n_type, "Neuron"]

            type_mask = np.array([lab in neurons_in_type.values for lab in labels_red])
            # ax.scatter(U_red[type_mask, eig_id], V_red[type_mask, eig_id], color=colors[i], s=80, label=n_type,
            #            zorder=3, edgecolor="k", lw=1, alpha=1, marker=markers[i])
            
            ax.scatter(U_red[type_mask, eig_id], V_red[type_mask, eig_id], color=color_red[type_mask], s=160, label=n_type,
                       zorder=3, edgecolor="k", lw=1, alpha=1, marker=markers[i])


        squared_rad = U_red[:, eig_id] ** 2 + V_red[:, eig_id] ** 2
        by_radius = np.argsort(squared_rad)
        alpha_vector = (squared_rad > np.percentile(squared_rad, 100-alpha_perc)).astype(int)
        
        for i, lab in enumerate(labels_red):
            ax.text(1.01*U_red[i, eig_id], 1.01*V_red[i, eig_id], 
                    lab, fontsize=12, alpha=0.8*alpha_vector[i],
                    path_effects=[
                        withStroke(
                            linewidth=4,
                            foreground="w",
                            alpha=0.8*alpha_vector[i]
                            )])

        ax.plot([0, 0], [y_min, y_max], ls=":", lw=2, color="k", zorder=0)
        ax.plot([x_min, x_max], [0, 0], ls=":", lw=2, color="k", zorder=0)

        ax.set_title(titles[ax_i*2 + ax_j], fontsize=22 * fontscale)
        ax.spines[["top", "right"]].set_visible(False)
        ax.spines[:].set_linewidth(2)
        
        ax.set_xticks(np.linspace(-0.2, .4, 4))
        ax.set_yticks(np.linspace(-.1, .2, 4))

        ax.set_xlim(prev_xlim)
        ax.set_ylim(prev_ylim)

        ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True, labelsize=16 * fontscale)

        ax.legend(handles=legend_elements, fontsize=18 * fontscale, loc="lower right")

axes[1, 0].set_xlabel(f"{eig_id + 1}th Left Singular Vector ($\\mathbf{{u}}_{{{eig_id+1}}}$)", fontsize=18 * fontscale)
axes[1, 1].set_xlabel(f"{eig_id + 1}th Left Singular Vector ($\\mathbf{{u}}_{{{eig_id+1}}}$)", fontsize=18 * fontscale)
axes[0, 0].set_ylabel(f"{eig_id + 1}th Right Singular Vector ($\\mathbf{{v}}_{{{eig_id+1}}}$)", fontsize=18 * fontscale)
axes[1, 0].set_ylabel(f"{eig_id + 1}th Right Singular Vector ($\\mathbf{{v}}_{{{eig_id+1}}}$)", fontsize=18 * fontscale)

# Overleaf version
fig.savefig(op.join(path_to_figures, f"Suppl-CElegans-SourcesSinks{eig_id:02d}.png"), dpi=100, bbox_inches="tight")
fig.savefig(op.join(path_to_figures, f"Suppl-CElegans-SourcesSinks{eig_id:02d}.pdf"), dpi=300, bbox_inches="tight", format="pdf")

## *C. elegans* Pathways

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(30, 20),
                         #sharex="row", sharey="row",
                         gridspec_kw={"wspace":0.12, "hspace":0.15})

node_size = 160

show_overlap = False
eig_id = 0

# titles = ["Right source ($u>0$), upper sink ($v>0$)",
#           "Left source ($u<0$), lower sink ($v<0$)",
#           "Left source ($u<0$), upper sink ($v>0$)",
#           "Connections with *AVE*"]

titles = ["Body Sensory to Motor",
          "Head Sensory to Motor",
          "Head Sensory to Body Motor",
          "Connections with *AVE*"]

zero_deg_thresh = 0
eig_thresh = 0

mask_list = []
for ax_i, ax in enumerate(axes.flatten()):

    wiring_red = wiring_sym.copy()

    invert_mask = False
    if ax_i == 0:
        sink = np.logical_and(wiring_red.sum(axis=1) <= zero_deg_thresh, V[:, eig_id] > eig_thresh, dtype=bool)
        source = np.logical_and(wiring_red.sum(axis=0) <= zero_deg_thresh, U[:, eig_id] > eig_thresh, dtype=bool)
    elif ax_i == 1:
        sink = np.logical_and(wiring_red.sum(axis=1) <= zero_deg_thresh, V[:, eig_id] < -eig_thresh, dtype=bool)
        source = np.logical_and(wiring_red.sum(axis=0) <= zero_deg_thresh, U[:, eig_id] < -eig_thresh, dtype=bool)
    elif ax_i == 2:
        sink = np.logical_and(wiring_red.sum(axis=1) <= zero_deg_thresh, V[:, eig_id] > eig_thresh, dtype=bool)
        source = np.logical_and(wiring_red.sum(axis=0) <= zero_deg_thresh, U[:, eig_id] < -eig_thresh, dtype=bool)
    #elif ax_i == 2:
    #    sink = np.array([lab in ["AVEL", "AVER"] for lab in nodes_labels])
    #    source = sink
    #    invert_mask = True
    else:
        sink = np.array([lab in ["AVEL", "AVER"] for lab in nodes_labels])
        #sink = np.array([lab in ["LUAL"] for lab in nodes_labels])
        source = sink

    sink_source = np.logical_or(sink, source, dtype=bool)
    #sink_source = np.logical_and(sink, source, dtype=bool)

    #mask = (wiring_red[sink_source].sum(axis=0) > 0) + (wiring_red[:, sink_source].sum(axis=1) > 0)
    mask = (wiring_red[source].sum(axis=0) > 0) + (wiring_red[:, sink].sum(axis=1) > 0)
    mask = (mask + sink_source).astype(bool)

    if invert_mask:
        mask = ~mask

    mask_list.append(mask)

    wiring_red = wiring_red[mask][:, mask]
    labels_red = nodes_labels[mask].to_list()
    sink_red = sink[mask]
    source_red = source[mask]
    U_red = U[mask]
    V_red = V[mask]

    color_red = ap_colors[mask]

    graph = nx.DiGraph(wiring_red)
    graph_pos = {i: (U_red[i, eig_id], V_red[i, eig_id]) for i in range(wiring_red.shape[0])}
    
    mycmap = plt.get_cmap("tab20")
    colors_edge = np.array(["tab:red", "tab:blue", "tab:green", "tab:gray"])
    colors = np.array(palette_rgb)

    edge_colors = np.zeros_like(np.array(graph.edges())[:, 0])

    for i, (_, target) in enumerate(graph.edges()):
        edge_colors[i] = neuron_df[neuron_df["Neuron"] == labels_red[target]]["Type_num"].values[0]
    #nx.draw_networkx_edges(graph, pos=graph_pos, alpha=.1, ax=ax)
    nx.draw_networkx_edges(graph, pos=graph_pos, edge_color=colors_edge[edge_colors],
                           alpha=2*edge_a, ax=ax, connectionstyle=edge_cs)

    ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)

    for i, n_type in enumerate(true_types): 
    # for i, n_type in enumerate(neuron_df["Type"].unique()):
        neurons_in_type = neuron_df.loc[neuron_df["Type"] == n_type, "Neuron"]

        type_mask = np.array([lab in neurons_in_type.values for lab in labels_red])
        # ax.scatter(U_red[type_mask, eig_id], V_red[type_mask, eig_id], color=colors[i], s=node_size, label=n_type,
        #                zorder=3, edgecolor="k", lw=1, alpha=1, marker=markers[i])
        
        ax.scatter(U_red[type_mask, eig_id], V_red[type_mask, eig_id], color=color_red[type_mask], s=node_size, label=n_type,
                    zorder=3, edgecolor="k", lw=1, alpha=1, marker=markers[i])

    squared_rad = U_red[:, eig_id] ** 2 + V_red[:, eig_id] ** 2
    by_radius = np.argsort(squared_rad)
    alpha_vector = (squared_rad > np.percentile(squared_rad, 100-alpha_perc)).astype(int)
    
    for i, lab in enumerate(labels_red):
        ax.text(1.01*U_red[i, eig_id], 1.01*V_red[i, eig_id], 
                lab, fontsize=12, alpha=0.8*alpha_vector[i],
                path_effects=[
                    withStroke(
                        linewidth=4,
                        foreground="w",
                        alpha=0.8*alpha_vector[i]
                        )])

    ax.plot([0, 0], [y_min, y_max], ls=":", lw=2, color="k", zorder=0)
    ax.plot([x_min, x_max], [0, 0], ls=":", lw=2, color="k", zorder=0)

    ax.set_title(titles[ax_i], fontsize=22 * fontscale)
    ax.spines[["top", "right"]].set_visible(False)
    ax.spines[:].set_linewidth(2)
    
    ax.set_xticks(np.linspace(-0.2, .4, 4))
    ax.set_yticks(np.linspace(-.1, .2, 4))

    ax.set_xlim(prev_xlim)
    ax.set_ylim(prev_ylim)

    ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True, labelsize=16 * fontscale)
    ax.legend(handles=legend_elements, fontsize=18 * fontscale, loc="lower right")

axes[1, 0].set_xlabel(f"{eig_id + 1}th Left Singular Vector ($\\mathbf{{u}}_{{{eig_id+1}}}$)", fontsize=18 * fontscale)
axes[1, 1].set_xlabel(f"{eig_id + 1}th Left Singular Vector ($\\mathbf{{u}}_{{{eig_id+1}}}$)", fontsize=18 * fontscale)
axes[0, 0].set_ylabel(f"{eig_id + 1}th Right Singular Vector ($\\mathbf{{v}}_{{{eig_id+1}}}$)", fontsize=18 * fontscale)
axes[1, 0].set_ylabel(f"{eig_id + 1}th Right Singular Vector ($\\mathbf{{v}}_{{{eig_id+1}}}$)", fontsize=18 * fontscale)

fig.savefig(op.join(path_to_figures, f"Suppl-CElegans-Pathways{eig_id:02d}.png"), dpi=100, bbox_inches="tight")
fig.savefig(op.join(path_to_figures, f"Suppl-CElegans-Pathways{eig_id:02d}.pdf"), dpi=300, bbox_inches="tight", format="pdf")

## *C. elegans* Bicommunities for k = 9

In [ ]:
graph = wiring_sym.copy()

U, S, Vh = dgsp.sorted_SVD(dgsp.modularity_matrix(graph, null_model="outin"))
V = Vh.T

n_nodes = graph.shape[0]

vector_id_max = 5
n_kmeans = 9

edge_clusters, edge_clusters_mat = dgsp.edge_bicommunities(graph, U, V, vector_id_max, method="kmeans",
                                                           n_kmeans=n_kmeans, verbose=True, max_k=10)
n_clusters = np.max(edge_clusters)

sending_communities, receiving_communities = dgsp.get_node_clusters(edge_clusters, edge_clusters_mat, method="bimodularity")

bimod_quad = dgsp.bimod_index_nodes(graph, sending_communities, receiving_communities, scale=True)
#bimod_quad = bimod_quad**sum_power/np.sum(bimod_quad**sum_power)
sorted_by_quad = np.flip(np.argsort(np.abs(bimod_quad)))

titles = [f"$Q_{{q}}(C_{{{i+1}}})={bimod_quad[i]:1.2f}$" for i in range(n_clusters)]

# Proportion of edges
titles = [title + f" ({100*(edge_clusters_mat == i+1).sum()/(graph > 0).sum():2.1f}%)" for i, title in enumerate(titles)]
titles = np.array(titles)[sorted_by_quad]

n_show_comp = 5
#n_show_comp = 3

x_norm = nodes_posx + 0.4
y_norm = (nodes_posy - nodes_posy.min())

angle = np.arctan2(y_norm/np.abs(y_norm).max(), x_norm/np.abs(x_norm).max())
angle = (angle - angle.min())/(angle.max() - angle.min())

angle = 1 - angle

fig, axes = plt.subplots(ncols=3, figsize=(15, 5*len(sending_communities[:n_show_comp])),
                         gridspec_kw={"wspace":0, "width_ratios":[2, 1, 3]})

letters = ["A.", "B.", "C."]
for ax, let in zip(axes, letters):
    ax.axis("off")
    ax.set_title(let, fontsize=22 * fontscale, loc="left", fontdict={"fontweight": "bold"})

graph_pos = {i: (x, y) for i, (x, y) in enumerate(zip(nodes_posx, nodes_posy))}

plot.plot_all_bicommunity(graph, sending_communities[sorted_by_quad][:n_show_comp],
                               receiving_communities[sorted_by_quad][:n_show_comp], gspec_hspace=0.3,
                               cmap=RedPurpleBlue, fig=fig, axes=axes[0], nrows=n_show_comp, draw_legend=False,
                               scatter_only=False, titles=titles[:n_show_comp], layout=graph_pos)

true_types = ["SENSORY NEURONS", "INTERNEURONS", "MOTOR NEURONS", "SEX SPECIFIC"]
manual_types = ["SENSORY", "INTER", "MOTOR", "SEX"]

type2lab = {t: l for t, l in zip(true_types, manual_types)}
type_mod = neuron_df.loc[:, "Type"].map(type2lab)

colors = {t: c for t, c in zip(manual_types, palette_rgb)}

n_bins = 60

count_weights = np.ones(n_bins)
count_weights[:47] = 3

gs_bars = GridSpecFromSubplotSpec(nrows=n_show_comp, ncols=1, subplot_spec=axes[2])
axes_bar = [fig.add_subplot(gs_bars[i], facecolor="none") for i in range(n_show_comp)]

for ax_i, com_id in enumerate(sorted_by_quad[:n_show_comp]):

    axes_bar[ax_i].axis("off")

    count, bins = np.histogram(angle, bins=n_bins)
    
    axes_bar[ax_i].text(0.05, count.max(), "Tail", fontsize=20)
    axes_bar[ax_i].text(0.9, count.max(), "Head", fontsize=20)
    
    axes_bar[ax_i].bar(bins[:-1], count_weights*count, width=1/n_bins, color="silver", alpha=1, edgecolor="gray", lw=2)
    axes_bar[ax_i].bar(bins[:-1], -count_weights*count, width=1/n_bins, color="silver", alpha=1, edgecolor="gray", lw=2)

    com_label = ["Send", "Receive"]
    mult = [1, -1]
    for com_i, com in enumerate([sending_communities[com_id], receiving_communities[com_id]]):
        fltr_com = com > 0
        count_previous = np.zeros_like(count, dtype=float)

        for i, n_type in enumerate(manual_types):
            #print(n_type)
            neurons_in_type = neuron_df.loc[type_mod == n_type, "Neuron"]
            type_mask = np.array([lab in neurons_in_type.values for lab in nodes_labels])

            mask = np.logical_and(type_mask, fltr_com)

            weights = np.ones_like(angle[mask])
            
            count_norm, _ = np.histogram(angle[mask], bins=bins, weights=mult[com_i]*weights)
            
            axes_bar[ax_i].bar(bins[:-1], count_weights*count_norm, width=1/n_bins, bottom=count_weights*count_previous, color=palette_rgb[i], lw=2, edgecolor="k")
            
            count_previous += count_norm.copy()
    #axes_bar[ax_i].legend(ncols=4)

pie_gs = GridSpecFromSubplotSpec(len(sending_communities[:n_show_comp]), 1, subplot_spec=axes[1], wspace=0, hspace=0.1)
pie_axes = [fig.add_subplot(pie_gs[i] ) for i in range(len(sending_communities[:n_show_comp]))]
plot.plot_bicommunity_types(sending_communities[sorted_by_quad][:n_show_comp],
                       receiving_communities[sorted_by_quad][:n_show_comp], axes=pie_axes, fig=fig,
                       titles=titles, type_colors=colors, fontsize=14,
                       types=type_mod)

## Colorbar
xpos = axes[0].get_position().bounds[0]
ypos = axes[0].get_position().bounds[1]
xsize = axes[0].get_position().bounds[2]
ysize = axes[0].get_position().bounds[3]

#cmap_ax = fig.add_axes([xpos+2*xsize/5, ypos*0.9+ysize/2, xsize/5, ysize/30])

# cmap_ax_1 = fig.add_axes([xpos*1.2, ypos+ysize*0.35, xsize*0.8, ysize/50])
# cmap_ax_2 = fig.add_axes([xpos*1.2, ypos+ysize*0.71, xsize*0.8, ysize/50])

size_mult = 50
offset = 1.06
if n_show_comp > 5:
    size_mult = 200
    offset = 1.02

cmap_axes = [fig.add_axes([xpos*1.2, ypos+ysize*offset*(i+1)/(n_show_comp), xsize*0.8, ysize/size_mult]) for i in range(n_show_comp - 1)]

# for cmap_ax in [cmap_ax_1, cmap_ax_2]:
for cmap_ax in cmap_axes:
    cmap_ax.spines[:].set_visible(False)

    norm = Normalize(vmin=-1, vmax=1)
    cbar = fig.colorbar(
        ScalarMappable(norm=norm, cmap=RedPurpleBlue.reversed()),
        cax=cmap_ax,
        orientation="horizontal",
        ticks=np.linspace(-1, 1, 5),
    )
    #cbar.ax.set_yticks([-1, 0, 1], labels=["Receive", "Both", "Send"])
    cbar.ax.set_xticks([-1, 0, 1], labels=["Send", "Both", "Receive"])
    cbar.ax.tick_params(labelsize=12*fontscale)

    cmap_ax.scatter([-0.9], [0.5], color="none", s=80, edgecolor="k", lw=2, marker="s")
    cmap_ax.scatter([0], [0.5], color="none", s=80, edgecolor="k", lw=2, marker="o")
    cmap_ax.scatter([0.9], [0.5], color="none", s=60, edgecolor="k", lw=2, marker="D")

fig.savefig(op.join(path_to_figures, f"Suppl-CElegans-Bicommunities-Nvec{vector_id_max}k{n_kmeans}-Show{n_show_comp}.png"), dpi=100, bbox_inches="tight")
fig.savefig(op.join(path_to_figures, f"Suppl-CElegans-Bicommunities-Nvec{vector_id_max}k{n_kmeans}-Show{n_show_comp}.pdf"), dpi=300, bbox_inches="tight", format="pdf")

## *C. elegans* Negative Bimodularity Embedding

In [ ]:
importlib.reload(plot)

fontscale = 1.2
alpha_perc = 88
show_markers = True
uneven_ratios = 1
testing = False
eig_id = 1

fig, all_axes = plt.subplots(figsize=(2*12, 10))
suffix = "Rect" + "_".join([f"{inch:1.0f}" for inch in fig.get_size_inches()])

gs1 = GridSpecFromSubplotSpec(1, 2, subplot_spec=all_axes, width_ratios=[1, 1 + uneven_ratios])

axes = [fig.add_subplot(gs1[0]), fig.add_subplot(gs1[1])]

for ax in axes:
    ax.spines[:].set_linewidth(2)

all_axes.set_xticks([])
all_axes.set_yticks([])
all_axes.spines[:].set_visible(False)


#################################################################################
###     First plot
#################################################################################

axes[0] = plot.plot_spectrum(wiring_sym, vector_id=eig_id,
                             show_n_eig=14,
                             write_s=True, fig=fig,
                             split_ax=False,
                             ax=axes[0], fontscale=fontscale, fix_negative=False,
                             override_title="Singular Values Spectrum", title_letter="A.")


#################################################################################
###     Second plot
#################################################################################
graph = nx.DiGraph(wiring_sym)
graph_pos = {i: (U[i, eig_id], V[i, eig_id]) for i in range(wiring_sym.shape[0])}

if not testing:
    edge_colors = np.zeros_like(np.array(graph.edges())[:, 0])

    for i, (_, target) in enumerate(graph.edges()):
        edge_colors[i] = neuron_df[neuron_df["Neuron"] == nodes_labels[target]]["Type_num"].values[0]
        
    colors = palette_rgb

    nx.draw_networkx_edges(graph, pos=graph_pos, ax=axes[1], edge_color=colors_edges[edge_colors],
                           alpha=edge_a, connectionstyle=edge_cs)

type_manual = ["SENSORY NEURONS", "INTERNEURONS", "MOTOR NEURONS", "SEX SPECIFIC"]

#for i, n_type in enumerate(neuron_df["Type"].unique()):
for i, n_type in enumerate(type_manual):
    neurons_in_type = neuron_df.loc[neuron_df["Type"] == n_type, "Neuron"]

    type_mask = np.array([lab in neurons_in_type.values for lab in nodes_labels])
    axes[1].scatter(U[type_mask, eig_id], V[type_mask, eig_id], color=colors[i], s=80, label=n_type,
                    zorder=3, edgecolor="k", lw=1, alpha=1, marker=markers[i])

alpha_factor = 0.2
alpha_threshold = 0.45
squared_rad = U[:, eig_id] ** 2 + V[:, eig_id] ** 2
alpha_vector = squared_rad
alpha_vector = np.clip(alpha_vector/(alpha_vector.max() * alpha_factor), a_min=0, a_max=1)

alpha_vector[alpha_vector < alpha_threshold] = 0
alpha_vector = (squared_rad > np.percentile(squared_rad, alpha_perc)).astype(int)

sorted_by_radius = np.argsort(squared_rad)

#for i, neuron in enumerate(nodes_labels):
for i in sorted_by_radius:
    #if neuron in label_filter:
    if True:
        axes[1].text(1.01*U[i, eig_id], 1.01*V[i, eig_id], nodes_labels[i], fontsize=16, alpha=alpha_vector[i],
                     path_effects=[
                         withStroke(
                             linewidth=4,
                             foreground="w",
                             alpha=0.8*alpha_vector[i],
                             )])

x_max, y_max = np.max(np.vstack([U[:, eig_id], V[:, eig_id]]), axis=1)
x_min, y_min = np.min(np.vstack([U[:, eig_id], V[:, eig_id]]), axis=1)

abs_min = max([x_min, y_min])
abs_max = min([x_max, y_max])

#axes.plot([abs_min, abs_max], [abs_min, abs_max], color="tab:orange")
axes[1].plot([0, 0], [y_min, y_max], ls=":", lw=2, color="k", zorder=0)
axes[1].plot([x_min, x_max], [0, 0], ls=":", lw=2, color="k", zorder=0)

legend_elements = [Line2D([0], [0], color="w", alpha=1, markersize=10, markeredgecolor="k", markerfacecolor=palette_rgb[i],
                          markeredgewidth=2, marker=markers[i], label=type) for i, type in enumerate(type_manual)]

# Parameters
axes[1].set_title(f"B.", fontsize=22 * fontscale, loc="left", fontdict={"fontweight": "bold"})
axes[1].set_title(f"Bimodularity Embedding", fontsize=22 * fontscale)
#axes[1].set_title(f"Bi-modularity embedding $s_{{{eig_id+1}}}={S[eig_id]:1.2f}$", fontsize=22 * fontscale)
axes[1].tick_params(left=True, bottom=True, labelleft=True, labelbottom=True, labelsize=16 * fontscale, width=2)
axes[1].set_xticks(np.linspace(-0.2, .4, 4))
axes[1].set_yticks(np.linspace(-.1, .2, 4))
axes[1].set_xlabel(f"{eig_id + 1}th Left Singular Vector $\\mathbf{{u}}_{{{eig_id+1}}}$", fontsize=18 * fontscale)
axes[1].set_ylabel(f"{eig_id + 1}th Right Singular Vector $\\mathbf{{v}}_{{{eig_id+1}}}$", fontsize=18 * fontscale)
axes[1].spines[["top", "right"]].set_visible(False)
#axes[1].legend(fontsize=16 * fontscale, loc="lower right")
axes[1].legend(handles=legend_elements, fontsize=16 * fontscale, loc="lower right")

fig.savefig(op.join(path_to_figures, f"Suppl-CElegans-NegativeEmbeddings-eig{eig_id:02d}-{'-Gap'*gap_junc}.png"), dpi=100, bbox_inches="tight")
fig.savefig(op.join(path_to_figures, f"Suppl-CElegans-NegativeEmbeddings-eig{eig_id:02d}-{'-Gap'*gap_junc}.pdf"), dpi=300, bbox_inches="tight", format="pdf")

## *C. elegans* With Electric Gap

In [ ]:
gap_junc = True

wiring_sym_gap = np.genfromtxt("./data/celegans_graph"+gap_junc*"_GAP"+".csv", delimiter=",")

wiring_mod_gap = dgsp.modularity_matrix(wiring_sym_gap, null_model="outin")
print(f"Asymmetric wiring matrix has shape {wiring_sym_gap.shape}")

d_mat = np.diag(wiring_sym_gap.sum(axis=1))

U, S, Vh = dgsp.sorted_SVD(wiring_mod_gap, fix_negative=False)
V = Vh.T

sort_idx = np.flip(np.argsort(S))
S = S[sort_idx]
U = U[:, sort_idx]
V = V[:, sort_idx]

In [ ]:
importlib.reload(plot)

fontscale = 1.2
alpha_perc = 88
show_markers = True
uneven_ratios = 1
testing = False
eig_id = 0

fig, all_axes = plt.subplots(figsize=(2*12, 10))
suffix = "Rect" + "_".join([f"{inch:1.0f}" for inch in fig.get_size_inches()])

gs1 = GridSpecFromSubplotSpec(1, 2, subplot_spec=all_axes, width_ratios=[1, 1 + uneven_ratios])

axes = [fig.add_subplot(gs1[0]), fig.add_subplot(gs1[1])]

for ax in axes:
    ax.spines[:].set_linewidth(2)

all_axes.set_xticks([])
all_axes.set_yticks([])
all_axes.spines[:].set_visible(False)


#################################################################################
###     First plot
#################################################################################

_, undir_s, _ = dgsp.sorted_SVD(dgsp.modularity_matrix(wiring_sym))
axes[0].plot(undir_s, lw=4, color="silver", marker="o", markersize=10)

axes[0] = plot.plot_spectrum(wiring_sym_gap, vector_id=eig_id,
                             show_n_eig=14,
                             write_s=True, fig=fig,
                             split_ax=False,
                             ax=axes[0], fontscale=fontscale, fix_negative=False,
                             override_title="Singular Values Spectrum", title_letter="A.")


#################################################################################
###     Second plot
#################################################################################
graph = nx.DiGraph(wiring_sym_gap)
graph_pos = {i: (U[i, eig_id], V[i, eig_id]) for i in range(wiring_sym_gap.shape[0])}

if not testing:
    edge_colors = np.zeros_like(np.array(graph.edges())[:, 0])

    for i, (_, target) in enumerate(graph.edges()):
        edge_colors[i] = neuron_df[neuron_df["Neuron"] == nodes_labels[target]]["Type_num"].values[0]
        
    colors = palette_rgb

    nx.draw_networkx_edges(graph, pos=graph_pos, ax=axes[1], edge_color=colors_edges[edge_colors],
                           alpha=edge_a, connectionstyle=edge_cs)

type_manual = ["SENSORY NEURONS", "INTERNEURONS", "MOTOR NEURONS", "SEX SPECIFIC"]

#for i, n_type in enumerate(neuron_df["Type"].unique()):
for i, n_type in enumerate(type_manual):
    neurons_in_type = neuron_df.loc[neuron_df["Type"] == n_type, "Neuron"]

    type_mask = np.array([lab in neurons_in_type.values for lab in nodes_labels])
    axes[1].scatter(U[type_mask, eig_id], V[type_mask, eig_id], color=colors[i], s=80, label=n_type,
                    zorder=3, edgecolor="k", lw=1, alpha=1, marker=markers[i])

alpha_factor = 0.2
alpha_threshold = 0.45
squared_rad = U[:, eig_id] ** 2 + V[:, eig_id] ** 2
alpha_vector = squared_rad
alpha_vector = np.clip(alpha_vector/(alpha_vector.max() * alpha_factor), a_min=0, a_max=1)

alpha_vector[alpha_vector < alpha_threshold] = 0
alpha_vector = (squared_rad > np.percentile(squared_rad, alpha_perc)).astype(int)

sorted_by_radius = np.argsort(squared_rad)

#for i, neuron in enumerate(nodes_labels):
for i in sorted_by_radius:
    #if neuron in label_filter:
    if True:
        axes[1].text(1.01*U[i, eig_id], 1.01*V[i, eig_id], nodes_labels[i], fontsize=16, alpha=alpha_vector[i],
                     path_effects=[
                         withStroke(
                             linewidth=4,
                             foreground="w",
                             alpha=0.8*alpha_vector[i],
                             )])

x_max, y_max = np.max(np.vstack([U[:, eig_id], V[:, eig_id]]), axis=1)
x_min, y_min = np.min(np.vstack([U[:, eig_id], V[:, eig_id]]), axis=1)

abs_min = max([x_min, y_min])
abs_max = min([x_max, y_max])

#axes.plot([abs_min, abs_max], [abs_min, abs_max], color="tab:orange")
axes[1].plot([0, 0], [y_min, y_max], ls=":", lw=2, color="k", zorder=0)
axes[1].plot([x_min, x_max], [0, 0], ls=":", lw=2, color="k", zorder=0)

legend_elements = [Line2D([0], [0], color="w", alpha=1, markersize=10, markeredgecolor="k", markerfacecolor=palette_rgb[i],
                          markeredgewidth=2, marker=markers[i], label=type) for i, type in enumerate(type_manual)]

# Parameters
axes[1].set_title(f"B.", fontsize=22 * fontscale, loc="left", fontdict={"fontweight": "bold"})
axes[1].set_title(f"Bimodularity Embedding", fontsize=22 * fontscale)
#axes[1].set_title(f"Bi-modularity embedding $s_{{{eig_id+1}}}={S[eig_id]:1.2f}$", fontsize=22 * fontscale)
axes[1].tick_params(left=True, bottom=True, labelleft=True, labelbottom=True, labelsize=16 * fontscale, width=2)
axes[1].set_xticks(np.linspace(-0.2, .4, 4))
axes[1].set_yticks(np.linspace(-.1, .2, 4))
axes[1].set_xlabel(f"{eig_id + 1}th Left Singular Vector $\\mathbf{{u}}_{{{eig_id+1}}}$", fontsize=18 * fontscale)
axes[1].set_ylabel(f"{eig_id + 1}th Right Singular Vector $\\mathbf{{v}}_{{{eig_id+1}}}$", fontsize=18 * fontscale)
axes[1].spines[["top", "right"]].set_visible(False)
#axes[1].legend(fontsize=16 * fontscale, loc="lower right")
axes[1].legend(handles=legend_elements, fontsize=16 * fontscale, loc="lower right")

fig.savefig(op.join(path_to_figures, f"Suppl-CElegans-WithElecGap-eig{eig_id:02d}{'-Gap'*gap_junc}.png"), dpi=100, bbox_inches="tight")
fig.savefig(op.join(path_to_figures, f"Suppl-CElegans-WithElecGap-eig{eig_id:02d}{'-Gap'*gap_junc}.pdf"), dpi=300, bbox_inches="tight", format="pdf")

In [ ]:
graph = wiring_sym_gap.copy()

U, S, Vh = dgsp.sorted_SVD(dgsp.modularity_matrix(graph, null_model="outin"))
V = Vh.T

n_nodes = graph.shape[0]

vector_id_max = 5
n_kmeans = 9

edge_clusters, edge_clusters_mat = dgsp.edge_bicommunities(graph, U, V, vector_id_max, method="kmeans",
                                                           n_kmeans=n_kmeans, verbose=True, max_k=10)
n_clusters = np.max(edge_clusters)

sending_communities, receiving_communities = dgsp.get_node_clusters(edge_clusters, edge_clusters_mat, method="bimodularity")

bimod_quad = dgsp.bimod_index_nodes(graph, sending_communities, receiving_communities, scale=True)
#bimod_quad = bimod_quad**sum_power/np.sum(bimod_quad**sum_power)
sorted_by_quad = np.flip(np.argsort(np.abs(bimod_quad)))

titles = [f"$Q_{{q}}(C_{{{i+1}}})={bimod_quad[i]:1.2f}$" for i in range(n_clusters)]

# Proportion of edges
titles = [title + f" ({100*(edge_clusters_mat == i+1).sum()/(graph > 0).sum():2.1f}%)" for i, title in enumerate(titles)]
titles = np.array(titles)[sorted_by_quad]

n_show_comp = 9
n_show_comp = 3

x_norm = nodes_posx + 0.4
y_norm = (nodes_posy - nodes_posy.min())

angle = np.arctan2(y_norm/np.abs(y_norm).max(), x_norm/np.abs(x_norm).max())
angle = (angle - angle.min())/(angle.max() - angle.min())

angle = 1 - angle

fig, axes = plt.subplots(ncols=3, figsize=(15, 5*len(sending_communities[:n_show_comp])),
                         gridspec_kw={"wspace":0, "width_ratios":[2, 1, 3]})

letters = ["A.", "B.", "C."]
for ax, let in zip(axes, letters):
    ax.axis("off")
    ax.set_title(let, fontsize=22 * fontscale, loc="left", fontdict={"fontweight": "bold"})

graph_pos = {i: (x, y) for i, (x, y) in enumerate(zip(nodes_posx, nodes_posy))}

plot.plot_all_bicommunity(graph, sending_communities[sorted_by_quad][:n_show_comp],
                               receiving_communities[sorted_by_quad][:n_show_comp], gspec_hspace=0.3,
                               cmap=RedPurpleBlue, fig=fig, axes=axes[0], nrows=n_show_comp, draw_legend=False,
                               scatter_only=False, titles=titles[:n_show_comp], layout=graph_pos)

true_types = ["SENSORY NEURONS", "INTERNEURONS", "MOTOR NEURONS", "SEX SPECIFIC"]
manual_types = ["SENSORY", "INTER", "MOTOR", "SEX"]

type2lab = {t: l for t, l in zip(true_types, manual_types)}
type_mod = neuron_df.loc[:, "Type"].map(type2lab)

colors = {t: c for t, c in zip(manual_types, palette_rgb)}

n_bins = 60

count_weights = np.ones(n_bins)
count_weights[:47] = 3

gs_bars = GridSpecFromSubplotSpec(nrows=n_show_comp, ncols=1, subplot_spec=axes[2])
axes_bar = [fig.add_subplot(gs_bars[i], facecolor="none") for i in range(n_show_comp)]

for ax_i, com_id in enumerate(sorted_by_quad[:n_show_comp]):

    axes_bar[ax_i].axis("off")

    count, bins = np.histogram(angle, bins=n_bins)
    
    axes_bar[ax_i].text(0.05, count.max(), "Tail", fontsize=20)
    axes_bar[ax_i].text(0.9, count.max(), "Head", fontsize=20)
    
    axes_bar[ax_i].bar(bins[:-1], count_weights*count, width=1/n_bins, color="silver", alpha=1, edgecolor="gray", lw=2)
    axes_bar[ax_i].bar(bins[:-1], -count_weights*count, width=1/n_bins, color="silver", alpha=1, edgecolor="gray", lw=2)

    com_label = ["Send", "Receive"]
    mult = [1, -1]
    for com_i, com in enumerate([sending_communities[com_id], receiving_communities[com_id]]):
        fltr_com = com > 0
        count_previous = np.zeros_like(count, dtype=float)

        for i, n_type in enumerate(manual_types):
            #print(n_type)
            neurons_in_type = neuron_df.loc[type_mod == n_type, "Neuron"]
            type_mask = np.array([lab in neurons_in_type.values for lab in nodes_labels])

            mask = np.logical_and(type_mask, fltr_com)

            weights = np.ones_like(angle[mask])
            
            count_norm, _ = np.histogram(angle[mask], bins=bins, weights=mult[com_i]*weights)
            
            axes_bar[ax_i].bar(bins[:-1], count_weights*count_norm, width=1/n_bins, bottom=count_weights*count_previous, color=palette_rgb[i], lw=2, edgecolor="k")
            
            count_previous += count_norm.copy()
    #axes_bar[ax_i].legend(ncols=4)

pie_gs = GridSpecFromSubplotSpec(len(sending_communities[:n_show_comp]), 1, subplot_spec=axes[1], wspace=0, hspace=0.1)
pie_axes = [fig.add_subplot(pie_gs[i] ) for i in range(len(sending_communities[:n_show_comp]))]
plot.plot_bicommunity_types(sending_communities[sorted_by_quad][:n_show_comp],
                       receiving_communities[sorted_by_quad][:n_show_comp], axes=pie_axes, fig=fig,
                       titles=titles, type_colors=colors, fontsize=14,
                       types=type_mod)

## Colorbar
xpos = axes[0].get_position().bounds[0]
ypos = axes[0].get_position().bounds[1]
xsize = axes[0].get_position().bounds[2]
ysize = axes[0].get_position().bounds[3]

#cmap_ax = fig.add_axes([xpos+2*xsize/5, ypos*0.9+ysize/2, xsize/5, ysize/30])

# cmap_ax_1 = fig.add_axes([xpos*1.2, ypos+ysize*0.35, xsize*0.8, ysize/50])
# cmap_ax_2 = fig.add_axes([xpos*1.2, ypos+ysize*0.71, xsize*0.8, ysize/50])

size_mult = 50
offset = 1.06
if n_show_comp > 5:
    size_mult = 200
    offset = 1.02

cmap_axes = [fig.add_axes([xpos*1.2, ypos+ysize*offset*(i+1)/(n_show_comp), xsize*0.8, ysize/size_mult]) for i in range(n_show_comp - 1)]

# for cmap_ax in [cmap_ax_1, cmap_ax_2]:
for cmap_ax in cmap_axes:
    cmap_ax.spines[:].set_visible(False)

    norm = Normalize(vmin=-1, vmax=1)
    cbar = fig.colorbar(
        ScalarMappable(norm=norm, cmap=RedPurpleBlue.reversed()),
        cax=cmap_ax,
        orientation="horizontal",
        ticks=np.linspace(-1, 1, 5),
    )
    #cbar.ax.set_yticks([-1, 0, 1], labels=["Receive", "Both", "Send"])
    cbar.ax.set_xticks([-1, 0, 1], labels=["Send", "Both", "Receive"])
    cbar.ax.tick_params(labelsize=12*fontscale)

    cmap_ax.scatter([-0.9], [0.5], color="none", s=80, edgecolor="k", lw=2, marker="s")
    cmap_ax.scatter([0], [0.5], color="none", s=80, edgecolor="k", lw=2, marker="o")
    cmap_ax.scatter([0.9], [0.5], color="none", s=60, edgecolor="k", lw=2, marker="D")

fig.savefig(op.join(path_to_figures, f"Suppl-CElegans-BicommunitiesWithGap-Nvec{vector_id_max}k{n_kmeans}-Show{n_show_comp}.png"), dpi=100, bbox_inches="tight")
fig.savefig(op.join(path_to_figures, f"Suppl-CElegans-BicommunitiesWithGap-Nvec{vector_id_max}k{n_kmeans}-Show{n_show_comp}.pdf"), dpi=300, bbox_inches="tight", format="pdf")